# Association Rules
on the basis of MLXtend by Sebastian Rashka http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

In [ ]:
import pandas as pd
import numpy as np

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori as apriori
from mlxtend.frequent_patterns import association_rules

In [4]:
data = pd.read_csv('dataset_for_test_20200729_cleaned.csv',sep=';', decimal=",")
data.head()

,order_id,_date,user_id,plu_id,category_id,parent_category_id,product_quantity,product_sum
0,5efc2687f59a7f82441fe4ca,2020-07-01 09:00:39+03,5efb12173849bd7ecc4cdb50,3383918,855.0,1105.0,1.0,49.9
1,5efc2687f59a7f82441fe4ca,2020-07-01 09:00:39+03,5efb12173849bd7ecc4cdb50,3489545,853.0,849.0,1.0,89.9
2,5efc2687f59a7f82441fe4ca,2020-07-01 09:00:39+03,5efb12173849bd7ecc4cdb50,2068955,839.0,834.0,1.0,99.9
3,5efc2687f59a7f82441fe4ca,2020-07-01 09:00:39+03,5efb12173849bd7ecc4cdb50,3435295,838.0,834.0,2.0,33.8
4,5efc2687f59a7f82441fe4ca,2020-07-01 09:00:39+03,5efb12173849bd7ecc4cdb50,3435297,838.0,834.0,2.0,31.8


In [6]:
data.shape

(332642, 8)

In [49]:
df = data.loc[:, ['order_id', 'plu_id']]
df.head()

,order_id,plu_id
0,5efc2687f59a7f82441fe4ca,3383918
1,5efc2687f59a7f82441fe4ca,3489545
2,5efc2687f59a7f82441fe4ca,2068955
3,5efc2687f59a7f82441fe4ca,3435295
4,5efc2687f59a7f82441fe4ca,3435297


In [50]:
df.shape

(332642, 2)

In [51]:
df2 = df.groupby('order_id').agg(lambda x: x.tolist())
df2.head()

,plu_id
order_id,
5efc2687f59a7f82441fe4ca,"[3383918, 3489545, 2068955, 3435295, 3435297, ..."
5efc26a0d9c459f11a8e122f,"[1860, 2139858, 3170135, 3341462, 45463, 36955..."
5efc26a4d9c45903d38e1247,"[2137027, 1536, 3661004, 3664384, 3674166, 368..."
5efc26b4d9c459748e8e126c,"[3477505, 43693, 79108, 3639749, 3444750, 3682..."
5efc26c1f59a7f62941fe522,"[2144368, 18510, 3480546, 3224596, 3998557, 40..."


In [52]:
df2.shape

(23773, 1)

In [58]:
temp = df2['plu_id'].to_list()
len(temp)

23773

In [56]:
te = TransactionEncoder()
te_ary = te.fit(temp).transform(temp)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [57]:
MIN_SUPPORT = 0.03

frequent_itemsets = apriori(df, min_support=MIN_SUPPORT, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.047659,(1536)
1,0.074791,(3757)
2,0.055441,(3759)
3,0.179027,(4072)
4,0.120052,(5013)
5,0.042611,(6177)
6,0.030497,(7827)
7,0.035166,(11034)
8,0.065789,(21292)
9,0.030329,(27768)


In [47]:
CONFIDENCE = 0.2

association_rules(frequent_itemsets, metric="confidence", min_threshold=CONFIDENCE)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(5013),(4072),0.120052,0.179027,0.033357,0.277856,1.552035,0.011865,1.136855
1,(45463),(4072),0.105203,0.179027,0.033820,0.321471,1.795663,0.014986,1.209932
2,(4072),(45505),0.179027,0.165398,0.057965,0.323778,1.957574,0.028354,1.234214
3,(45505),(4072),0.165398,0.179027,0.057965,0.350458,1.957574,0.028354,1.263926
4,(4072),(3638820),0.179027,0.107433,0.040382,0.225564,2.099581,0.021149,1.152538
5,(3638820),(4072),0.107433,0.179027,0.040382,0.375881,2.099581,0.021149,1.315412
6,(4072),(3639749),0.179027,0.129264,0.040382,0.225564,1.744982,0.017240,1.124348
7,(3639749),(4072),0.129264,0.179027,0.040382,0.312398,1.744982,0.017240,1.193966
8,(3695580),(4072),0.066630,0.179027,0.032684,0.490530,2.739985,0.020756,1.611427
9,(45505),(45463),0.165398,0.105203,0.045514,0.275178,2.615677,0.028113,1.234505


In [48]:
LIFT = 1.2


rules = association_rules(frequent_itemsets, metric="lift", min_threshold=LIFT)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(4072),(5013),0.179027,0.120052,0.033357,0.186325,1.552035,0.011865,1.081449
1,(5013),(4072),0.120052,0.179027,0.033357,0.277856,1.552035,0.011865,1.136855
2,(4072),(45463),0.179027,0.105203,0.033820,0.188910,1.795663,0.014986,1.103202
3,(45463),(4072),0.105203,0.179027,0.033820,0.321471,1.795663,0.014986,1.209932
4,(4072),(45505),0.179027,0.165398,0.057965,0.323778,1.957574,0.028354,1.234214
5,(45505),(4072),0.165398,0.179027,0.057965,0.350458,1.957574,0.028354,1.263926
6,(4072),(3638820),0.179027,0.107433,0.040382,0.225564,2.099581,0.021149,1.152538
7,(3638820),(4072),0.107433,0.179027,0.040382,0.375881,2.099581,0.021149,1.315412
8,(4072),(3639749),0.179027,0.129264,0.040382,0.225564,1.744982,0.017240,1.124348
9,(3639749),(4072),0.129264,0.179027,0.040382,0.312398,1.744982,0.017240,1.193966
